In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1) 데이터 가져오기

2) EDA, preprocessing - 빈칸 등 처리하고 데이터 다듬기(cleaning)

3) tokenization - 어떤 거 쓸지? 여러가지 써보자

4) one-hot encoding

5) word embedding - word2vec 써보기 GloVe도 써보고 싶다! 5번 생략하고 one-hot vector로 바로 돌려보고 비교해보는 것도?

6) modelling - LSTM + BRNN / training - optimizer 선택

7) evaluation

Q. 2,3,5,6 번의 선택지를 조합하면 너무 많은데 (한개의 모델 내에서 하이퍼파라미터를 조정하는 걸 차치하고) 이 중에서 어떻게 가장 좋은 성능을 내는 걸 찾아내지? 다해보기엔 너무 많은 것 같은데..

# 1. 데이터 가져오기

In [ ]:
df_train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

print('training set shape = {}'.format(df_train.shape))
print('test set shape = {}'.format(df_test.shape))

# 2-1. EDA

In [ ]:
df_train.head()

In [ ]:
import pandas_profiling
# 데이터프레임을 .profile_report() 한 줄로 탐색할 수 있음!

pr = df_train.profile_report()
pr

keyword 컬럼의 비어있는 셀 0.8프로와 location의 33프로를 어떻게 처리할 것인가?

셀 데이터 형태를 살펴보고, 살려서 가지고 갈 수 있는 방법이 있는지도 공부해야함. 일단 지금은 모델을 끝까지 짜보는게 목표라서 location은 빼버리고 keyword null 값을 가지는 example은 제외하는 방향으로 진행

참고한 노트북은 location은 남아있는 데이터들도 너무 지저분해서 feature로 사용하지 않음. keyword는 데이터가 깨끗하고 output과 명확한 상관관계가 나타난다고 판단.(밑의 그래프에서 확인 가능)

In [ ]:
df_train['target_mean'] = df_train.groupby('keyword')['target'].transform('mean')
# target_mean 컬럼을 추가할건데 target 컬럼을 keyword 필터로 정렬하는 컬럼. 뒤에 transform은 뭐더라?

import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(8,72))
sns.countplot(y=df_train.sort_values(by='target_mean', ascending = False)['keyword'],
             hue = df_train.sort_values(by='target_mean', ascending = False)['target'])

plt.legend(loc=1)
plt.show()

df_train.drop(columns=['target_mean'], inplace = True)

다음으로 텍스트에 대한 EDA 도 진행. 해당 노트북은 disaster tweet이 non-disaster tweet 보다 긴 단어(단어 한개의 평균 길이가 더 길다는 건지, 트윗의 길이가 더 길다는 건지는 잘 모르겠다)를 사용하고, 타이포가 적을 것라고 추측하여 진행함(공식 news agencies 가 올린 트윗들이 있기 때문에). 다음과 같은 meta features 사용

* `word_count` number of words in text
* `unique_word_count` number of unique words in text
* `stop_word_count` number of stop words in text
* `url_count` number of urls in text
* `mean_word_length` average character count in words
* `char_count` number of characters in text
* `punctuation_count` number of punctuations in text
* `hashtag_count` number of hashtags (**#**) in text
* `mention_count` number of mentions (**@**) in text

In [ ]:
# word_count
df_train['word_count'] = df_train['text'].apply(lambda x: len(str(x).split()))
df_test['word_count'] = df_test['text'].apply(lambda x: len(str(x).split()))

# unique_word_count, set이 무슨 기능이지?
df_train['unique_word_count'] = df_train['text'].apply(lambda x: len(set(str(x).split())))
df_test['unique_word_count'] = df_test['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count, wordcloud의 STOPWORDS 사용 (nltk.corpus의 stopwords 도 한번 써보기)
from wordcloud import STOPWORDS
df_train['stop_word_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
df_test['stop_word_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))

# url_count
df_train['url_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
df_test['url_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
df_train['mean_word_length'] = df_train['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test['mean_word_length'] = df_test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# character_count
df_train['char_count'] = df_train['text'].apply(lambda x: len(str(x)))
df_test['char_count'] = df_test['text'].apply(lambda x: len(str(x)))

# punctuation_count, string의 punctuation 사용
import string
df_train['punctuation_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test['punctuation_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
df_train['hashtag_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
df_test['hashtag_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
df_train['mention_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
df_test['mention_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

In [ ]:
METAFEATURES = ['word_count', 'unique_word_count', 'stop_word_count', 'url_count', 'mean_word_length',
                'char_count', 'punctuation_count', 'hashtag_count', 'mention_count']
disaster_tweets = df_train['target'] == 1

# 이 밑으로는 제대로 이해 못했다. 파이플롯 공부 좀 더해
fig, axes = plt.subplots(ncols=2, nrows=len(METAFEATURES), figsize=(20, 50), dpi=100)

for i, feature in enumerate(METAFEATURES):
    sns.distplot(df_train.loc[~disaster_tweets][feature], label='Not Disaster', ax=axes[i][0], color='green')
    sns.distplot(df_train.loc[disaster_tweets][feature], label='Disaster', ax=axes[i][0], color='red')

    sns.distplot(df_train[feature], label='Training', ax=axes[i][1])
    sns.distplot(df_test[feature], label='Test', ax=axes[i][1])
    
    for j in range(2):
        axes[i][j].set_xlabel('')
        axes[i][j].tick_params(axis='x', labelsize=12)
        axes[i][j].tick_params(axis='y', labelsize=12)
        axes[i][j].legend()
    
    axes[i][0].set_title(f'{feature} Target Distribution in Training Set', fontsize=13)
    axes[i][1].set_title(f'{feature} Training & Test Set Distribution', fontsize=13)

plt.show()


똑같이 복사 붙여넣기 했는데 왜 에러나냐고?????ㅜㅜ
You have categorical data, but your model needs something numerical. See our one hot encoding tutorial for a solution. 

어쨌든 2-1까지는 다음 노트북을 참고함
https://www.kaggle.com/gunesevitan/nlp-with-disaster-tweets-eda-cleaning-and-bert (이 노트북에서는 embedding 후에 text cleaning을 하던데 이게 어떻게 되지?)

In [ ]:
df_train.drop(columns=['word_count', 'unique_word_count', 'stop_word_count', 'url_count', 'mean_word_length', 
                       'char_count', 'punctuation_count', 'hashtag_count', 'mention_count'], inplace = True)
df_test.drop(columns=['word_count', 'unique_word_count', 'stop_word_count', 'url_count', 'mean_word_length', 
                      'char_count', 'punctuation_count', 'hashtag_count', 'mention_count'], inplace = True)

# 2-2. 데이터 전처리

트윗이라서 이상한 줄임말 + 대문자 남용 + @,# 등의 특수문자가 너무 많은데 이 부분 어떻게 처리할지.
stemmization 같은 걸 해야하나? 해도 의미가 없는지 아니면 트윗이라서 오히려 더 필요한지

In [ ]:
df_total = pd.concat([df_train, df_test])
df_total.shape

In [ ]:
# url 제거
# 정규표현식 모둘 re 사용
import re

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

example="New competition launched :https://www.kaggle.com/c/nlp-getting-started"
remove_URL(example)

In [ ]:
df_total['text'] = df_total['text'].apply(lambda x: remove_URL(x))

In [ ]:
# html 제거

def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'',text)

In [ ]:
df_total['text'] = df_total['text'].apply(lambda x: remove_html(x))

In [ ]:
# 이모지(emojis) 제거
# 여기 코드는 이해 못함.. 위에도 이해는 하는데 직접 쓰라면 못쓸듯. 나중에 re 직접 작성해보기

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

remove_emoji("Omg another Earthquake 😔😔")

In [ ]:
df_total['text'] = df_total['text'].apply(lambda x: remove_emoji(x))

In [ ]:
# punctuation 제거

def remove_punct(text):
    table = str.maketrans('','',string.punctuation)
    return text.translate(table)

print(remove_punct("I am a #king"))

In [ ]:
df_total['text'] = df_total['text'].apply(lambda x: remove_punct(x))

In [ ]:
#spellchecker를 이용해서 스펠링 커렉션

!pip install pyspellchecker
#어떤 패키지에 이걸 진행하는 거지? 바로 임포트 할 수 있는 패키지와 뭐가 다르죵

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spelling(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split()) # text를 스페이스로 split해서 그 중 스펠체커가 인식못하는 애들 리스트
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word)) # text의 단어가 misspelled_words 에 속하는 애면 correction 진행 후 append
        else:
            corrected_text.append(word) # 그게 아니면 변경없이 그대로 append
    return ' '.join(corrected_text)

correct_spelling('corect me plese')

In [ ]:
# 이 셀 돌리는데 너무 오래 걸리는디

df_total['text'] = df_total['text'].apply(lambda x: correct_spelling(x))

2-2 중 여기까지는 다음 노트북을 참고함 https://www.kaggle.com/shahules/basic-eda-cleaning-and-glove

In [ ]:
df_total['text'].head(20)

# 지금까지 한거 중간확인. 전부 소문자로 변환하고 불용어 제거할 것. 
# 다른 노트북 보면 숫자도 다 없애던데 숫자는 남겨두는게 더 낫지 않나?
# 스페이스가 두개인 부분도 처리해줘야 하나?
# 저 gooooooooaaaaaaal 같은건 어떻게허냐...;_; - spellchecker가 제대로 돌아갔으면 처리됐겠지?

> 앞에서 정제한 데이터들로 1. 소문자로 변경, 2. 불용어 제거를 진행한다. 근데 소문자로 변경을 스펠체크보다 먼저 해야할거 같은데.. 순서 잘못한것 같다

In [ ]:
df_total['text'] = df_total['text'].apply(lambda x: x.lower())

In [ ]:
df_total['text'].head(10)

In [ ]:
df_total['text'] = df_total['text'].apply(lambda x: correct_spelling(x))

In [ ]:
def delete_stopwords(text):
    result = []
    for word in text.split():
        if word not in STOPWORDS:
            result.append(word)
    return result

delete_stopwords('I am coming')

In [ ]:
df_total['text_stopwords'] = df_total['text'].apply(lambda x: delete_stopwords(str(x)))

In [ ]:
df_total['text_stopwords'].head(10)

왜 저렇게 나와....? 일단 이거 말고 다른 notebook에서 가져온거 쓰자. 소문자화, 불용어처리, lemmatization 한번에 진행하는거

In [ ]:
def massage_text(text):
    from nltk.corpus import stopwords
    ## remove anything other then characters and put everything in lowercase
    tweet = re.sub("[^a-zA-Z]", ' ', text)
    tweet = tweet.lower()
    tweet = tweet.split()

    from nltk.stem import WordNetLemmatizer
    lem = WordNetLemmatizer()
    tweet = [lem.lemmatize(word) for word in tweet
             if word not in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    return tweet

In [ ]:
df_total['clean_text'] = df_total['text'].apply(lambda x: massage_text(x))

In [ ]:
df_total.head(10)

여기서는 참고하지 않았지만 cleaning 에 대해 쉽게 잘 써놓은 노트북 https://www.kaggle.com/vanshjatana/a-simple-guide-to-text-cleaning 나중에 꼭 보기!

# 3. tokenization

In [ ]:
from nltk.tokenize import word_tokenize

df_total['tokens']= df_total['clean_text'].apply(lambda x: word_tokenize(x))

In [ ]:
df_total.head()

# 5. Word2Vec (word embedding)

4. one-hot encoding을 미리 진행해놓을 필요 없이 패키지가 거기서부터 한꺼번에 처리해주는 것 같다.

In [ ]:
import gensim
def fn_pre_process_data(text):
    for word in text:
        yield gensim.utils.simple_preprocess(word)
        

In [ ]:
corpus = list(fn_pre_process_data(df_total['clean_text']))

# 이 함수 역할은 잘 모르겠지만 왜 tokens 컬럼이 아니라 clean_text를 이용해서 corpus를 만들지? 
# 내가 아는 word2vec은 token 을 one-hot vector로 만든 다음에 LM을 통해서 embedding matrix 를 구하는건뎅..

In [ ]:
from gensim.models import Word2Vec

In [ ]:
wv_model = Word2Vec(corpus, size = 150, window = 3, min_count = 2)
wv_model.train(corpus, total_examples = len(corpus), epochs=10)
# 요 부분이 뉴럴넷 LM로 학습시키는 부분인듯? 

In [ ]:
def get_word_embeddings(token_list, vector, k=150):
    if len(token_list) < 1: #토큰 리스트가 없을경우
        return np.zeros(k)
    else:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in token_list]
        
    sum = np.sum(vectorized, axis=0)
    return sum/len(vectorized) #몽가... 평균값을 리턴

def get_embeddings(token,vector):
    embeddings = token.apply(lambda x: get_word_embeddings(x, wv_model))
    return list(embeddings)


In [ ]:
df_total['tokens']

In [ ]:
total_embeddings = get_embeddings(df_total['tokens'], wv_model)


In [ ]:
total_embeddings

In [ ]:
type(total_embeddings)

In [ ]:
len(total_embeddings)

여기까지는 다음 노트북을 참고함. https://www.kaggle.com/slatawa/simple-implementation-of-word2vec

https://www.kaggle.com/parthplc/word2vec-tutorial 에서 Word2Vec from scratch 를 다루고 있는 듯 하니 나중에 한번 보자

In [ ]:
train_embeddings = total_embeddings[:7613]
test_embeddings = total_embeddings[7613:]

In [ ]:
x_total = train_embeddings
y_total = df_train['target']

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_total, y_total, test_size = 0.3)

# 6. modelling


In [ ]:
# We will use the official tokenization script created by the Google team
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import tokenization

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
model = build_model(bert_layer, max_len=160)
model.summary()

In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    X_train, Y_train,
    validation_split=0.2,
    epochs=3,
    callbacks=[checkpoint],
    batch_size=16
)

....? 난 왜 샘플이 8개야?

In [ ]:
model.load_weights('model.h5')
test_pred = model.predict(test_embeddings)

In [ ]:
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)

다들 이렇게 제출하던데 왜 난 안되는겨. 한번에 되는게 하나도 없냐!!!

In [ ]:
predictions = test_pred.round().astype(int)
predictions

In [ ]:
output = pd.DataFrame({'id': df_test['id'], 'target': predictions})
output.to_csv('submission.csv', index = False)

모델링 부분에서 참고한 노트북 https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub 하나도 이해못하고 그냥 갖다 붙여넣기한거ㅜㅜ

LSTM은 하지도 못했는데?

챕터마다 서로 다른 노트북을 참고하다 보니 output하고 input이 잘 안이어지는 것 같다. output은 매트릭스를 줬는데 다음 input에는 1D 리스트를 넣어줘야 한다든지, 얘는 딕셔너리를 줬는데 리스트로 변경해서 모델에 넣어야 한다든지. 그래서 뒤로 갈수록 제대로 된 결과가 안나오는 듯함.
